In [1]:
import pandas as pd
import data_scraper as ds

In [2]:
all_countries = ds.wiki_shortcodes.keys()
all_countries

dict_keys(['Austria', 'Belgium', 'Denmark', 'France', 'Germany', 'Italy', 'Japan', 'the_Netherlands', 'New_York_(state)', 'South_Korea', 'Spain', 'the_United_Kingdom', 'the_United_States'])

In [3]:
def get_data(download=True):
    if download:
        all_countries = ds.wiki_shortcodes.keys()
        df = ds.download_data(all_countries)
        df.to_pickle('wikipedia_data.pickle')
        return df
    return pd.read_pickle('wikipedia_data.pickle')

In [4]:
wiki_data = get_data(download=False)

In [5]:
wiki_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 80 entries, 0 to 79
Data columns (total 27 columns):
date         80 non-null datetime64[ns]
cases_at     80 non-null int64
deaths_at    80 non-null int64
cases_be     80 non-null int64
deaths_be    80 non-null int64
cases_dk     80 non-null int64
deaths_dk    80 non-null int64
cases_fr     80 non-null int64
deaths_fr    80 non-null int64
cases_de     80 non-null int64
deaths_de    80 non-null int64
cases_it     80 non-null int64
deaths_it    80 non-null int64
cases_jp     80 non-null int64
deaths_jp    80 non-null int64
cases_nl     80 non-null int64
deaths_nl    80 non-null int64
cases_ny     80 non-null int64
deaths_ny    80 non-null int64
cases_kr     80 non-null int64
deaths_kr    80 non-null int64
cases_es     80 non-null int64
deaths_es    80 non-null int64
cases_uk     80 non-null int64
deaths_uk    80 non-null int64
cases_us     80 non-null int64
deaths_us    80 non-null int64
dtypes: datetime64[ns](1), int64(26)
memory usage: 

In [6]:
wiki_data

,date,cases_at,deaths_at,cases_be,deaths_be,cases_dk,deaths_dk,cases_fr,deaths_fr,cases_de,...,cases_ny,deaths_ny,cases_kr,deaths_kr,cases_es,deaths_es,cases_uk,deaths_uk,cases_us,deaths_us
0,2020-01-21,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,2020-01-22,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,2020-01-23,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
3,2020-01-24,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,2,0
4,2020-01-25,0,0,0,0,0,0,0,0,0,...,0,0,2,0,0,0,0,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,2020-04-05,11907,204,19691,1447,4561,179,70478,8078,91714,...,122031,4159,10284,186,131646,12641,47806,5373,333593,9534
76,2020-04-06,12206,220,20814,1632,4875,187,74390,8911,95391,...,130689,4758,10331,192,136675,13341,51608,6159,362955,10748
77,2020-04-07,12519,243,22194,2035,5266,203,78167,10328,99225,...,138836,5489,10384,200,141942,14045,55242,7097,393707,12674
78,2020-04-08,12852,273,23403,2240,5597,218,82048,10869,103228,...,149316,6268,10423,204,148220,14792,60733,7978,425746,14610
